# IMDB 영화 리뷰 감성 분석 - Conv1D
    - 참고: https://velog.io/@changdaeoh/Convolutionforsequence, https://sevity.tistory.com/58, https://yjjo.tistory.com/8

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
import warnings
warnings.filterwarnings('ignore')

### Conv1D로 IMDB 리뷰 감성 분석
- 단어 빈도수: 10,000 (88,584)
- 문장의 단어수: 500 (2,494)
- test data 중 40%(10,000개)는 검증용으로

In [2]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
X_train.shape, X_test.shape, y_train.shape

17473536/17464789 [==============================] - 0s 0us/step


((25000,), (25000,), (25000,))

In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 500), (25000, 500))

In [6]:
from sklearn.model_selection import train_test_split

X_test, X_valid, y_test, y_valid = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.4, random_state=seed
)
X_test.shape, X_valid.shape, y_test.shape, y_valid.shape

((15000, 500), (10000, 500), (15000,), (10000,))

- Case 1) Conv1D x 2, MaxPooling1D x 2, Dropour, GlobalMaxPooling1D
    - embedding dim: 100
    - hidden_units: 128

In [7]:
model1 = Sequential([ 
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

In [8]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best_imdb-conv1d.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [9]:
hist1 = model1.fit(
    X_train, y_train, epochs=30, batch_size=128,
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
196/196 [==============================] - ETA: 0s - loss: 0.5406 - accuracy: 0.7002
Epoch 00001: val_loss improved from inf to 0.35781, saving model to best_imdb-conv1d.h5
196/196 [==============================] - 19s 20ms/step - loss: 0.5406 - accuracy: 0.7002 - val_loss: 0.3578 - val_accuracy: 0.8420
Epoch 2/30
196/196 [==============================] - ETA: 0s - loss: 0.2818 - accuracy: 0.8860
Epoch 00002: val_loss improved from 0.35781 to 0.31920, saving model to best_imdb-conv1d.h5
196/196 [==============================] - 4s 18ms/step - loss: 0.2818 - accuracy: 0.8860 - val_loss: 0.3192 - val_accuracy: 0.8630
Epoch 3/30
193/196 [============================>.] - ETA: 0s - loss: 0.2073 - accuracy: 0.9207
Epoch 00003: val_loss did not improve from 0.31920
196/196 [==============================] - 3s 16ms/step - loss: 0.2074 - accuracy: 0.9206 - val_loss: 0.3345 - val_accuracy: 0.8605
Epoch 4/30
196/196 [==============================] - ETA: 0s - loss: 0.1552 - accur

In [10]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 2s 3ms/step - loss: 0.3168 - accuracy: 0.8661


[0.3167656362056732, 0.8660666942596436]

Case 2) Conv1D + LSTM

In [11]:
from tensorflow.keras.layers import LSTM

In [12]:
model2 = Sequential([ 
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                      

In [13]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best_imdb-conv1d-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [14]:
hist2 = model2.fit(
    X_train, y_train, epochs=30, batch_size=128,
    validation_data=[X_valid, y_valid],
    callbacks=[mc, es]
)

Epoch 1/30
195/196 [============================>.] - ETA: 0s - loss: 0.4301 - accuracy: 0.7854
Epoch 00001: val_loss improved from inf to 0.30102, saving model to best_imdb-conv1d-lstm.h5
196/196 [==============================] - 8s 27ms/step - loss: 0.4299 - accuracy: 0.7855 - val_loss: 0.3010 - val_accuracy: 0.8705
Epoch 2/30
196/196 [==============================] - ETA: 0s - loss: 0.2163 - accuracy: 0.9192
Epoch 00002: val_loss improved from 0.30102 to 0.28939, saving model to best_imdb-conv1d-lstm.h5
196/196 [==============================] - 5s 24ms/step - loss: 0.2163 - accuracy: 0.9192 - val_loss: 0.2894 - val_accuracy: 0.8796
Epoch 3/30
196/196 [==============================] - ETA: 0s - loss: 0.1570 - accuracy: 0.9437
Epoch 00003: val_loss did not improve from 0.28939
196/196 [==============================] - 6s 32ms/step - loss: 0.1570 - accuracy: 0.9437 - val_loss: 0.3142 - val_accuracy: 0.8828
Epoch 4/30
196/196 [==============================] - ETA: 0s - loss: 0.121

In [15]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 3s 6ms/step - loss: 0.2883 - accuracy: 0.8805


[0.28834468126296997, 0.880466639995575]

Case 3) Conv1D + Dense

In [17]:
model3 = Sequential([ 
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_3 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_4 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)              

In [18]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best_imdb-conv1d-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [19]:
hist3 = model3.fit(
    X_train, y_train, epochs=30, batch_size=128,
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
194/196 [============================>.] - ETA: 0s - loss: 0.5696 - accuracy: 0.6855
Epoch 00001: val_loss improved from inf to 0.35787, saving model to best_imdb-conv1d-lstm.h5
196/196 [==============================] - 4s 18ms/step - loss: 0.5684 - accuracy: 0.6864 - val_loss: 0.3579 - val_accuracy: 0.8440
Epoch 2/30
194/196 [============================>.] - ETA: 0s - loss: 0.2963 - accuracy: 0.8763
Epoch 00002: val_loss improved from 0.35787 to 0.27440, saving model to best_imdb-conv1d-lstm.h5
196/196 [==============================] - 3s 17ms/step - loss: 0.2961 - accuracy: 0.8764 - val_loss: 0.2744 - val_accuracy: 0.8845
Epoch 3/30
192/196 [============================>.] - ETA: 0s - loss: 0.2033 - accuracy: 0.9222
Epoch 00003: val_loss improved from 0.27440 to 0.26129, saving model to best_imdb-conv1d-lstm.h5
196/196 [==============================] - 3s 16ms/step - loss: 0.2035 - accuracy: 0.9219 - val_loss: 0.2613 - val_accuracy: 0.8919
Epoch 4/30
193/196 [=========

In [20]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 3s 5ms/step - loss: 0.2599 - accuracy: 0.8943


[0.2599080204963684, 0.8943333625793457]